In [1]:
import pandas as pd
import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Download the stopwords from NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lhmtr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lhmtr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Merge text and label df
text_df = pd.read_json("raw_data/train_text.json",lines=True)
label_df = pd.read_json("raw_data/train_truth.json", lines=True)

df = pd.merge(text_df,label_df, on='twitter user id').drop(columns = ['tweet ids'])

In [3]:
# Expand the dataframe
expanded_rows = []
for index, row in df.iterrows():
    for text_dict in row['texts']:
        new_row = row.copy()
        new_row['texts'] = text_dict['text']
        expanded_rows.append(new_row)

df = pd.DataFrame(expanded_rows)

In [4]:
def preprocess_tweet(tweet):
    # Translate emojis to words
    tweet = emoji.demojize(tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    # Remove user mentions and count them
    mentions = re.findall(r'@\w+', tweet)
    count_mentions = len(mentions)
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove special characters and numbers
    tweet = re.sub(r'\W', ' ', tweet)

    # Tokenize and remove stopwords
    tokens = word_tokenize(tweet)
    tokens = [word.lower() for word in tokens if word.lower() not in stopwords.words('english')]

    # Join the words back into a single string
    tweet = ' '.join(tokens)

    return tweet, count_mentions

In [5]:
# Apply the preprocessing function to the 'tweet' column
df['texts'], df['count_mention'] = zip(*df['texts'].apply(preprocess_tweet))

# Remove duplicate tweets based on the 'tweet_processed' column
df = df.drop_duplicates(subset=['texts'])

def count_words(text):
    return len(text.split())

# Count the number of words in each row of the specified column
df['word_count'] = df['texts'].apply(count_words)

# Remove rows with less than 5 words in the specified column
df = df[df['word_count'] >= 5]

# Drop the 'word_count' column as it's no longer needed
df = df.drop(columns=['word_count'])

# Save the preprocessed DataFrame to a new CSV file
df.to_csv('data/preprocessed_data.csv', index=False)

In [6]:
df

,twitter user id,texts,class,count_mention
0,0037a672f0ed64b3231bac64853a278d,rt ape ape give apes tweeting great pics atamc...,nano,1
0,0037a672f0ed64b3231bac64853a278d,see single valid reason sell ape,nano,1
1,03eaa72711143b521c073d9ac5745923,rt need knock heels together three times say s...,nano,1
1,03eaa72711143b521c073d9ac5745923,index_pointing_up would love input tezos clean...,nano,0
1,03eaa72711143b521c073d9ac5745923,rt fresh drop purple_heart 3 men please merrie...,nano,1
...,...,...,...,...
159,fec182516cba4b665e2215094bbcc527,rt ok giving away 0 5 sol 5 discord invites rt...,nano,1
159,fec182516cba4b665e2215094bbcc527,rt sol vs eth sol handshake eth,nano,1
159,fec182516cba4b665e2215094bbcc527,rt wrapped_gift vanguards x yaku corp wrapped_...,nano,2
159,fec182516cba4b665e2215094bbcc527,rt giving away y00ts t00b nft crystal_ball flo...,nano,1
